## Детектирование сигнала светофора

Проект направлен на обнаружение сигнала светофора с использованием машинного обучения
Задачи:
- Обнаружение красного (круглого) сигнала в дневное время
- Обнаружение желтого (круглого) сигнала в дневное время
- Обнаружение зеленого (круглого) сигнала в дневное время
- Детектирование отсутствия светофора в дневное время

Для проекта используются следующие датасеты:
- тренировочный (4 датасета: red, yellow, green, empty - без светофора)
-  тестовый  (4 датасета: red, yellow, green, empty - без светофора)

Каждый датасет представляет собой набор изображений с трехсекционным светофором в дневное время (для набора empty - без светофора в дневное время).

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from google.colab import files
%matplotlib inline 

## Загружаем данные

In [ ]:
!wget https://www.dropbox.com/s/85srzvwk1owrr5t/train_cut.zip?dl=0  -O train.zip

In [ ]:
!wget https://www.dropbox.com/s/p7kniyf6fige4eq/test_cut.zip?dl=0 -O test.zip

In [ ]:
!ls

In [ ]:
!unzip -q train.zip

In [ ]:
!ls train

cat  dog


In [ ]:
!unzip -q test.zip

In [ ]:
!ls test

cat  dog


Смотрим примеры картинок

In [ ]:
img_path = 'train/cat/cat.0.jpg'
img = image.load_img(img_path, target_size=(224, 224))
plt.imshow(img)
plt.show()

## Создаем наборы данных

Набор данных для обучения

In [ ]:
train_dataset = image_dataset_from_directory('train',
                                             batch_size=128,
                                             image_size=(224, 224))

In [ ]:
class_names = train_dataset.class_names

In [ ]:
class_names

Набор данных для тестирования

In [ ]:
test_dataset = image_dataset_from_directory('test',
                                             batch_size=128,
                                             image_size=(224, 224))

## Создаем составную нейронную сеть

In [ ]:
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
inputs = layers.Input(shape=(224, 224, 3))
x = img_augmentation(inputs)
model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

# Freeze the pretrained weights
model.trainable = False

# Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
# Для задачи с двумя классами изображений
outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)
# Для задачи с несколькими классами изображений
# num_classes = 3 # Задаем количество классов
# outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)
model = tf.keras.Model(inputs, outputs, name="EfficientNet")

Компилируем составную нейронную сеть

In [ ]:
# Для задачи с двумя классами изображений
model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])
# Для задачи с несколькими классами изображений
# model.compile(loss='categorical_crossentropy',
#              optimizer='adam', 
#              metrics=['accuracy'])

## Обучаем сеть

In [ ]:
history = model.fit(train_dataset,
                    epochs=10)

Проверяем качество обучения на тестовом наборе данных


In [ ]:
scores = model.evaluate(test_dataset, verbose=1)

In [ ]:
print("Доля верных ответов на тестовых данных, в процентах:", round(scores[1] * 100, 4))

## Использование нейронной сети для распознавания изображений

Вспоминаем номера классов

In [ ]:
class_names

Загружаем изображение из файла

In [ ]:
img_path = 'test/dog/dog.1000.jpg'
img = image.load_img(img_path, target_size=(224, 224))
plt.imshow(img)
plt.show()

Преобразуем изображение в массив

In [ ]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

Запускаем распознавание

In [ ]:
prediction = model.predict(x)

Печатаем результаты распознавания

In [ ]:
prediction


array([[0.9814932]], dtype=float32)